## Demo for SDK (latest)

This example shows how to use SDK to deploy a task. The notebook includes the following steps:
- [initialization](#initialization)
- [get `job_source_uri` (not covered in this demo)](#get-job_source_uri)
- [show available hardware info list](#show-available-hardware-information)
- [choose hardware config](#choose-hardware-config)
- [estimate payment amount](#estimate-payment-amount)
- [create task to get `task_uuid`](#create-task)
- [submit payment](#submit-payment)
- [validate payment](#validate-payment-to-deploy-task)
- [follow up task status](#follow-up-task-status-optional)
- [show result](#show-result)



### Initialization

- For test version, get `API_KEY` in dashboard page: https://orchestrator-test.swanchain.io
- For prod version, get `API_KEY` in dashboard page: https://orchestrator.swanchain.io

In [1]:
import os
import time
import dotenv
dotenv.load_dotenv("../.env")
from swan import SwanAPI , MCSAPI

# dev_url = "https://swanhub-cali.swanchain.io"
dev_url = "http://127.0.0.1:5008"

# Initialize the Swan Service
swan_api = SwanAPI(api_key=os.getenv("API_KEY"), environment=dev_url)
# for prod version
# swan_api = SwanAPI(api_key=os.getenv("API_KEY"))

# api_key = os.getenv("MCS_API_KEY")
# mcs_api = MCSAPI(api_key)


from swan.contract.swan_contract import SwanContract

contract = SwanContract(os.getenv('PK'), os.getenv('RPC'))

### Get `job_source_uri`
to simplify the process, here we use a existing `job_source_uri` which is a hello world application, used to create task.

In [2]:
job_source_uri = 'https://test-api.lagrangedao.org/spaces/5117e998-c623-4837-8af9-2b7b0ce2de7f'

### Show available hardware information

In [3]:
hardwares = swan_api.get_hardware_config()
hardwares_info = [hardware.to_dict() for hardware in hardwares if hardware.status == "available"] 
hardwares_info

[{'id': 0,
  'name': 'C1ae.small',
  'description': 'CPU only · 2 vCPU · 2 GiB',
  'type': 'CPU',
  'reigion': ['North Carolina-US', 'Quebec-CA'],
  'price': '0.0',
  'status': 'available'},
 {'id': 1,
  'name': 'C1ae.medium',
  'description': 'CPU only · 4 vCPU · 4 GiB',
  'type': 'CPU',
  'reigion': ['North Carolina-US', 'Quebec-CA'],
  'price': '1.0',
  'status': 'available'},
 {'id': 12,
  'name': 'G1ae.small',
  'description': 'Nvidia 3080 · 4 vCPU · 8 GiB',
  'type': 'GPU',
  'reigion': ['North Carolina-US', 'Quebec-CA'],
  'price': '10.0',
  'status': 'available'},
 {'id': 13,
  'name': 'G1ae.medium',
  'description': 'Nvidia 3080 · 8 vCPU · 16 GiB',
  'type': 'GPU',
  'reigion': ['North Carolina-US', 'Quebec-CA'],
  'price': '11.0',
  'status': 'available'},
 {'id': 20,
  'name': 'Hpc1ae.small',
  'description': 'Nvidia 3090 · 4 vCPU · 8 GiB',
  'type': 'GPU',
  'reigion': ['Quebec-CA'],
  'price': '14.0',
  'status': 'available'},
 {'id': 21,
  'name': 'Hpc1ae.medium',
  'desc

### Choose hardware config

choose the hardware with its config name or hardware id

in hardware config, `cfg_name` and `hardware_id` will be used in the steps to deploy task (create task and submit payment)

In [4]:
cfg_name = 'C1ae.medium' #"G1ae.medium"
hardware = [hardware for hardware in hardwares if hardware.name == cfg_name][0]
print(f"{hardware.name=}, {hardware.id=}, {hardware.region}")

hardware_id = hardware.id
print(f"The chosen {hardware_id=}")

hardware.name='C1ae.medium', hardware.id=1, ['North Carolina-US', 'Quebec-CA']
The chosen hardware_id=1


### Estimate Payment amount

In [5]:
duration_hour = 1 # hour
amount = contract.estimate_payment(hardware_id, duration_hour)
print(amount)

1000000000000000000


### Create task

This step uses above information to create task, and get `task_uuid`, which is useful in submit payment step.

In [6]:
import json
duration=3600*duration_hour

result = swan_api.create_task(
    cfg_name=cfg_name, 
    region='Quebec-CA', 
    start_in=5, 
    duration=duration, 
    job_source_uri=job_source_uri, 
    paid=contract._wei_to_swan(amount),
    wallet_address=os.getenv('WALLET'),
)
print(json.dumps(result, indent=2))
task_uuid = result['data']['task']['uuid']

{
  "data": {
    "task": {
      "created_at": "1713391986",
      "end_at": "1713395583",
      "leading_job_id": null,
      "refund_amount": null,
      "status": "initialized",
      "task_detail_cid": "https://plutotest.acl.swanipfs.com/ipfs/QmPEEFw6akm3h5CatzKnk3hXPHKnYjGjSb8aG5tPXCX1rh",
      "tx_hash": null,
      "updated_at": "1713391986",
      "uuid": "328c1665-cf9a-4e9d-ab6b-9b42b3d844b4"
    }
  },
  "message": "Task_uuid initialized.",
  "status": "success"
}


### Submit Payment

This step is using `task_uuid`, `hardware_id` and `duration` to submit payment via **ClientPayment** contract.

In [7]:
tx_hash = contract.submit_payment(task_uuid, hardware_id, duration)
print(tx_hash)

0x2d53f92083a99bf4e5cc636364586262639b79d0d96a3d8c36a1c8b69a1ef17c


### Validate Payment to deploy task

This step will validate the payment and then make task eligible for assigning if validation successful

In [8]:
if result_validation := swan_api.validate_payment(
    tx_hash=tx_hash,
    task_uuid=task_uuid
):
    print(json.dumps(result_validation, indent=2))
else:
    print('validation failed')

{'tx_hash': '0x2d53f92083a99bf4e5cc636364586262639b79d0d96a3d8c36a1c8b69a1ef17c', 'task_uuid': '328c1665-cf9a-4e9d-ab6b-9b42b3d844b4'}
{
  "data": {
    "task": {
      "created_at": "1713391986",
      "end_at": "1713395583",
      "leading_job_id": null,
      "refund_amount": null,
      "status": "created",
      "task_detail_cid": "https://plutotest.acl.swanipfs.com/ipfs/QmPEEFw6akm3h5CatzKnk3hXPHKnYjGjSb8aG5tPXCX1rh",
      "tx_hash": null,
      "updated_at": "1713392006",
      "uuid": "328c1665-cf9a-4e9d-ab6b-9b42b3d844b4"
    }
  },
  "message": "Task payment validated successfully.",
  "status": "success"
}


### follow up task status (optional)
The following step is optional, shows information when waiting for task being deployed.

In [10]:
# Check task info
info = swan_api.get_deployment_info(task_uuid=task_uuid)
print(json.dumps(info, indent=2))

{
  "data": {
    "computing_providers": [
      {
        "allowed_nodes": null,
        "autobid": 1,
        "city": "Montreal",
        "country": null,
        "created_at": "1712735800",
        "deleted_at": null,
        "last_active_at": null,
        "lat": 45.5075,
        "lon": -73.5887,
        "multi_address": "/ip4/38.140.46.60/tcp/8086",
        "name": "cp-2.44-2",
        "node_id": "0478a886cb6e191a0531fb6f8a7da4c0fbf0ec220af6d49e622f7e2da514f0ea205ed77918796c66ae642f0996dbf0ecc4bb7f1dd949705d1306abd1c14b60fb8a",
        "online": true,
        "public_address": "0xFbc1d38a2127D81BFe3EA347bec7310a1cfa2373",
        "region": "Quebec-CA",
        "score": 100,
        "status": "Active",
        "updated_at": "1713370970"
      }
    ],
    "jobs": [
      {
        "bidder_id": "0478a886cb6e191a0531fb6f8a7da4c0fbf0ec220af6d49e622f7e2da514f0ea205ed77918796c66ae642f0996dbf0ecc4bb7f1dd949705d1306abd1c14b60fb8a",
        "build_log": "wss://log.dev2.crosschain.computer:

### Show result

`job_real_uri` is for show the result of application you deployed.

In [11]:
r = swan_api.get_real_url(task_uuid)
print(r)

['https://m6z09emb8i.dev2.crosschain.computer']


In [12]:
import requests
import json

headers = {
    'Content-Type': 'application/json',
}

response = requests.get(r[0], headers=headers)

try:
    print(json.dumps(response.json(), indent=4))
except Exception as e:
    print(e)
    print(response)


{
    "Hello": "World! Today - 06.21"
}
